<a href="https://colab.research.google.com/github/nssn96/ML_Neural-networks/blob/main/ML_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 2 - Neural Networks


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873



In [15]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [16]:
#References Used
#https://towardsdatascience.com/https-medium-com-piotr-skalski92-deep-dive-into-deep-networks-math-17660bc376ba
#https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

# This is the Layer class

class Layer:

  def initial_w_bias(self,nn):
    
    #random seed initiation
    np.random.seed(90)
    n_layers = len(nn)
    param = dict()

    for index,layer in enumerate(nn):
      layer_num = index+1
      lyr_inputsize = layer['d_input']
      lyr_outputsize = layer['d_output']

      #creating the values for the w and bias matrix
      param['w'+str(layer_num)] = np.random.randn(lyr_outputsize,lyr_inputsize) * 0.1
      param['b'+str(layer_num)] = np.random.randn(lyr_outputsize,1) * 0.1


  def forward(self):
    pass


  def backward(self):
    pass



In [17]:
# This is the Linear layer class

class Linearlayer(Layer):

  # def forward_single(self,w,bias,prev_out,activ='tanh_forwardprop'):
  #   # Z_mat--> the input matrix calculation which will be passed in the activation function
  #   Z_mat = np.dot(w,prev_out)
  #   return activ(Z_mat),Z_mat # we are returning the calculated z value here because we need the z-value during backward pass calculation

 #this is the full forward function to calculate whole step forward which will make use of forward_single function to eliminate iteration and speed up calculation 
  def forward(self, data, param ,nn):
    backup=dict() # to store info required for the backward calculation
    A_current = data # the input data is the activation for layer 0 ie the input layer

    #traversing over the layers in the neural network
    #this is the same steps that we used to assign the weights and bias for network
    for index,lyr in enumerate(nn):
      lyr_num = index+1
      #just assigning the previous A value for the calculation of next Z value, since the output of layer(l-1) is the input for layer l
      A_before = A_current
      # getting the w and bias values from param for sending it to single forward function
      w = param['w'+str(lyr_num)]
      bias = param['b'+str(lyr_num)]

      #calculation of activation value A for the current layer
      # A_current, Z_current = self.forward_single(w,bias,A_before)

      # Z_mat--> the input matrix calculation which will be passed in the activation function
      Z_mat = np.dot(w,A_before) + bias
      backup['Z'+str(lyr_num)] = Z_mat
      A_current = self.activ(Z_mat)
      #storing the values for backward pass calc
      backup['A'+str(index)] = A_current
      

      return A_current,backup


  
  def backward_single(self):
    pass
  
  def backward(self):
    pass
  

In [18]:
#This is the class for the sigmoid function

class sigmoid_activation(Layer):

  def sigmoid_forwardprop(self,Z):
   return 1/(1+np.exp(-Z))
  
  def sigmoid_backprop(self,dA,z):
    value = self.sigmoid(z)
    return dA * value * (1-value)
  

In [19]:
#This is the class for the hyperbolic tangent functions

class htangent_activation(Layer):

  def tanh_forwardprop(self,z):
    #2/(1 + e-2x) - 1
    return (2/(1+np.exp(-2*z)) -1)
  
  def tanh_backprop(self,dA,z):
    value = self.tanh_forwardprop(z)
    return dA * value * (1-value)


In [20]:
#This is the class for the softmax function

class softmax_activation(Layer):

  def softmax_forwardprop(self,z):
    return np.exp(z) / sum(np.exp(z))

  
  def softmax_backprop(self,dA,z):
    value = self.softmax_forwardprop(z)
    return dA * value * (1-value)

  

In [21]:
# I will be providing the a list of dictionary that contains the parameters for the Neural network( ie the number of layers, number of nodes and activation function)
#d_input--> size of the input vector to the layer ie n(l-1) where l is layer
#d_ouput--> size of the activation vector at the output of the layer ie n(l)
nn = [
      {'d_input':2 ,'d_output':3, 'activation': 'tanh' } ,
       {'d_input':3 ,'d_output':1, 'activation': 'sigmoid' }
]

print(nn)

[{'d_input': 2, 'd_output': 3, 'activation': 'tanh'}, {'d_input': 3, 'd_output': 1, 'activation': 'sigmoid'}]


In [22]:
#testing
net = Linearlayer()
param = net.initial_w_bias(nn)
#print(net.forward())